In [17]:
import torch
from torch.nn import functional as F

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.device(device)

with open("datasets/tiny_shakespeare.txt", "r", encoding="utf-8") as file:
    text = file.read()

print("-- dataset len:", len(text))

# here are all the unique characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(f"-- dataset has {vocab_size} unique characters:", itos)

# let's now encode the entire text dataset and store it into a torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)

# let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

inputs, targets = get_batch("train")
print("-- inputs:", inputs)
print("-- targets:", targets)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = inputs[b, :t+1]
        target = targets[b, t]
        # print(f"-- when input is {context.tolist()} the target: {target}")

class BigramLanguageModel(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # create embeding table which is a tensor of shape [64, 64] filled with random numbers
        self.token_embedding_table = torch.nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        # we cant just train a model on indices of characters from lookup table, before we need to embed them. inputs shape is [4, 8]
        # after embeding we should get a tensor with shape [4, 8, 64], which means that each of 8 numbers in 4 rows now represented
        #  as an array of 64 random numbers. but why we need 64 numbers? Because we have so many unique characters in dataset and each
        #  number indicates a probability for that number (but for now probabilities are random)
        logits = self.token_embedding_table(idx)
        # now when we have predictions about what comes next, we would like to measure a loss (quality of predictions) using
        # cross_entropy (negative log likelihood), in order to do that we need to reshape logits and targets (only because
        #  cross_entropy function expects data in different shapes)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_idx):
        # idx is [B, T] tensor of indices
        for _ in range(max_new_idx):
            # get the predictions
            logits, loss = self(idx) # forward pass
            # focus only on the last time step
            logits = logits[:, -1, :] # logits now has shape of [B, C]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # [B, C]
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # [B, 1]
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # [B, T+1]
        return idx

model = BigramLanguageModel(vocab_size) # initializa the model

# tipical training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
batch_size = 32
steps = 10000
for step in range(steps):
    x, y = get_batch("train") # sample a batch of data
    logits, loss = model(x, y) # forward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if step+1 == steps:
        print("-- loss:", loss.item())

# generate a sample
idx = torch.zeros((1, 1), dtype=torch.long) # [[0]] - it is a reasonable starting context because 0 maps to a new line character
print(decode(model.generate(idx, 300)[0].tolist()))

# BigramLanguageModel is a very simple model because the tokens are not talking to each other so given the previous context of
# what was generated we are only looking at the very last character to make a predictions about what comes next.
# So now these tokens (characters) have to start talking to each other and figuring out what is in the context so that they
# can make better predictions for what comes next.

-- dataset len: 1115394
-- dataset has 65 unique characters: {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
-- inputs: tensor([[ 1, 47, 52,  1, 58, 46, 43,  1],
        [59, 56,  1, 54, 56, 47, 52, 41],
        [56,  8,  1, 18, 56, 47, 39, 56],
        [58, 43, 52, 57, 47, 53,  6,  1]])
-- targets: tensor([[47, 52,  1, 58, 46, 43,  1, 41],
        [56,  1, 54, 56, 47, 52, 41, 43],
        [ 8,  1, 18, 56, 47, 39, 56,  1],
        [43, 52, 57, 47, 53,  6,  1, 40]])
-